## Scraper para Al Calor Político

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time

In [20]:


# Configuración
base_url = "https://www.alcalorpolitico.com/edicion/inicio.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_article_links(main_url):
    print("🔍 Obteniendo enlaces...")
    try:
        response = requests.get(main_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        return list({
            urljoin(base_url, link['href']) if not link['href'].startswith('http') else link['href']
            for link in soup.select('a[href*="/informacion/"]')
        })
    except Exception as e:
        print(f"❌ Error: {e}")
        return []

def extract_article_content(article_body):
    if not article_body:
        return ""
    
    # Extraer primer párrafo (después de google-ads y antes del primer <br>)
    google_ads = article_body.find('div', class_='google-ads')
    first_para = []
    if google_ads:
        next_node = google_ads.next_sibling
        while next_node and getattr(next_node, 'name', None) != 'br':
            if getattr(next_node, 'string', None) and next_node.string.strip():
                first_para.append(next_node.string.strip())
            next_node = next_node.next_sibling
    
    # Extraer resto del contenido
    rest_content = [
        text.strip() for text in article_body.find_all(text=True)
        if text.parent.name not in ['script', 'style', 'ins', 'div']
        and text.strip() and text not in first_para
    ]
    
    return ' '.join(first_para + rest_content)

def extract_article_data(url):
    try:
        print(f"📄 Procesando: {url}")
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Título
        title = soup.find('h1').get_text(strip=True) if soup.find('h1') else 'Sin título'
        
        # Fecha - Extracción mejorada
        date = 'Sin fecha'
        date_element = soup.find('span', id='lugar')  # Nuevo selector
        if date_element:
            # Limpieza del texto (ej: "México / 25-Jun-2023" -> "25-Jun-2023")
            date_text = date_element.get_text(strip=True)
            if ' / ' in date_text:
                date = date_text.split(' / ')[-1]  # Tomar la parte después de la barra
            else:
                date = date_text
        
        # Contenido
        article_content = extract_article_content(soup.find('div', class_='cuerponota') or 
                                                soup.find('div', class_='cuerpoColumna'))
        
        return {
            'titulo': title,
            'fecha': date,
            'articulo': article_content,
            'url': url
        }
    
    except Exception as e:
        print(f"❌ Error en {url}: {e}")
        return None

# Ejecución principal
print("🚀 Iniciando scraping...")
links = get_article_links(base_url)[:500]  # Solo 3 para prueba
datos = [extract_article_data(link) for link in links if link is not None]
df = pd.DataFrame([d for d in datos if d])

# Mostrar resultados
df.head()


🚀 Iniciando scraping...
🔍 Obteniendo enlaces...
📄 Procesando: https://www.alcalorpolitico.com/informacion/noticias.php?idnota=419146


/tmp/ipykernel_56918/1327817022.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text.strip() for text in article_body.find_all(text=True)


📄 Procesando: https://www.alcalorpolitico.com/informacion/sheinbaum-recuerda-a-morena-evitar-nepotismo-en-elecciones-de-veracruz-y-durango-423904.html
📄 Procesando: https://alcalorpolitico.com/informacion/cuarta-edicion-de-factor-x-xalapa-415543.html
📄 Procesando: https://www.alcalorpolitico.com/informacion/sicario-dispara-a-pareja-el-hombre-murio-en-martinez-de-la-torre-423765.html
📄 Procesando: https://www.alcalorpolitico.com/informacion/menor-que-aprendia-a-manejar-atropello-y-mato-a-su-madre-en-coatzacoalcos-423641.html
📄 Procesando: https://www.alcalorpolitico.com/informacion/noticias.php?idnota=423856
📄 Procesando: https://www.alcalorpolitico.com/informacion/no-dejaremos-irregularidades-garantizan-autoridades-de-coatzacoalcos-423859.html
📄 Procesando: https://www.alcalorpolitico.com/informacion/despliegan-modulos-de-vacunacion-en-playas-de-la-zona-centro-de-veracruz-423761.html
📄 Procesando: https://www.alcalorpolitico.com/informacion/noticias.php?idnota=402271
📄 Procesando: http

,titulo,fecha,articulo,url
0,Entrevista con Agustín Basilio de la Vega.,24/01/2025,,https://www.alcalorpolitico.com/informacion/no...
1,Sheinbaum recuerda a MORENA evitar nepotismo e...,"Xalapa, Ver. 22/04/2025",La presidenta Claudia Sheinbaum Pardo recordó ...,https://www.alcalorpolitico.com/informacion/sh...
2,Cuarta edición de FACTOR X Xalapa,12/11/2024,,https://alcalorpolitico.com/informacion/cuarta...
3,"Sicario dispara a pareja; el hombre murió, en ...","Martínez de la Torre, Ver. 18/04/2025",Un hombre perdió la vida y una mujer resultó l...,https://www.alcalorpolitico.com/informacion/si...
4,Menor que aprendía a manejar atropelló y mató ...,"Coatzacoalcos, Ver. 16/04/2025",Una menor de edad atropelló y mató por acciden...,https://www.alcalorpolitico.com/informacion/me...


In [21]:
df = df[df['articulo'].str.len() > 50]  # Elimina filas con menos de 50 caracteres

In [29]:
import re

# Suponiendo que tu DataFrame es 'df' y la columna se llama 'fecha'
# Ejemplo de datos: "Orizaba, Ver. 22/04/2025"

# 1. Extraer lugar y fecha usando expresiones regulares
df[['lugar', 'fecha_limpia']] = df['fecha'].str.extract(
    r'^(.*?)\s*\.\s*(\d{2}/\d{2}/\d{4})$'  # Patrón: "texto. dd/mm/aaaa"
)

# 2. Limpieza adicional (opcional)
df['lugar'] = df['lugar'].str.strip()  # Elimina espacios en blanco
df['fecha_limpia'] = pd.to_datetime(df['fecha_limpia'], format='%d/%m/%Y')  # Convertir a datetime

# Verificar filas donde la extracción falló
filas_fallidas = df[df['fecha_limpia'].isna()]
if not filas_fallidas.empty:
    print("⚠️ Filas con formato inesperado:")
    display(filas_fallidas[['fecha']])
    
    # Opción: Extraer manualmente (ej: si algunas no tienen punto)
    df['fecha_limpia'] = df['fecha'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0]  # Solo fecha
    df['lugar'] = df['fecha'].str.replace(r'\d{2}/\d{2}/\d{4}', '', regex=True).str.strip(' .')

# 3. Mostrar resultados
print(df[['fecha', 'lugar', 'fecha_limpia']].head())

⚠️ Filas con formato inesperado:


,fecha
8,20/02/2024
16,21/04/2025
60,Ciudad de México 17/04/2025
77,Ciudad del Vaticano 21/04/2025
104,El Vaticano 22/04/2025
114,Ciudad de México 21/04/2025
140,Nueva York 19/04/2025
168,18/04/2025
179,17/04/2025
181,Ciudad de México 21/04/2025


                              fecha               lugar fecha_limpia
46          Xalapa, Ver. 02/05/2008         Xalapa, Ver   02/05/2008
200         Xalapa, Ver. 05/08/2019         Xalapa, Ver   05/08/2019
4    Coatzacoalcos, Ver. 16/04/2025  Coatzacoalcos, Ver   16/04/2025
223         Xalapa, Ver. 16/04/2025         Xalapa, Ver   16/04/2025
283    Ciudad de México. 16/04/2025    Ciudad de México   16/04/2025


In [30]:
df = df.sort_values(by='fecha_limpia')
df

,titulo,fecha,articulo,url,lugar,fecha_limpia
46,Atento aviso a nuestros lectores,"Xalapa, Ver. 02/05/2008","Es comprensible que en muchos casos, las perso...",https://www.alcalorpolitico.com/informacion/At...,"Xalapa, Ver",02/05/2008
200,Te invitamos a compartir el libro que leíste,"Xalapa, Ver. 05/08/2019",Te invitamos a seguir participando en la secci...,https://www.alcalorpolitico.com/informacion/no...,"Xalapa, Ver",05/08/2019
259,Tour por las mejores cafeterías,09/02/2024,acompáñame a mi si si sí Wachisneris en este t...,https://www.alcalorpolitico.com/informacion/no...,,09/02/2024
4,Menor que aprendía a manejar atropelló y mató ...,"Coatzacoalcos, Ver. 16/04/2025",Una menor de edad atropelló y mató por acciden...,https://www.alcalorpolitico.com/informacion/me...,"Coatzacoalcos, Ver",16/04/2025
223,No era tornado: “Embudo de nube” sorprendió en...,"Xalapa, Ver. 16/04/2025","La tarde de este miércoles, usuarios de redes ...",https://www.alcalorpolitico.com/informacion/no...,"Xalapa, Ver",16/04/2025
...,...,...,...,...,...,...
338,"Por obra que no avanza, no pueden usar cancha ...","Xalapa, Ver. 22/04/2025",Vecinos de la colonia Revolución se manifestar...,https://www.alcalorpolitico.com/informacion/po...,"Xalapa, Ver",22/04/2025
314,Exbasquetbolista “El Diablo” Castellanos dirig...,"Veracruz, Ver. 22/04/2025",Las Rojas de Veracruz han anunciado a su entre...,https://www.alcalorpolitico.com/informacion/ex...,"Veracruz, Ver",22/04/2025
322,Captan a presunto personal de PC de Tantoyuca ...,"Xalapa, Ver. 22/04/2025",Ciudadanos del municipio de Tantoyuca denuncia...,https://www.alcalorpolitico.com/informacion/ca...,"Xalapa, Ver",22/04/2025
208,Orizabeños también reciben llamadas de estafas...,"Orizaba, Ver. 22/04/2025",En el último mes han incrementado las llamadas...,https://www.alcalorpolitico.com/informacion/or...,"Orizaba, Ver",22/04/2025


In [31]:
# Guardar a CSV
df.to_csv('noticias_alcalorpolitico.csv', index=False, encoding='utf-8-sig')
print("💾 Datos guardados en 'noticias_alcalorpolitico.csv'")

# Mostrar en el notebook
from IPython.display import display
display(df)

💾 Datos guardados en 'noticias_alcalorpolitico.csv'


,titulo,fecha,articulo,url,lugar,fecha_limpia
46,Atento aviso a nuestros lectores,"Xalapa, Ver. 02/05/2008","Es comprensible que en muchos casos, las perso...",https://www.alcalorpolitico.com/informacion/At...,"Xalapa, Ver",02/05/2008
200,Te invitamos a compartir el libro que leíste,"Xalapa, Ver. 05/08/2019",Te invitamos a seguir participando en la secci...,https://www.alcalorpolitico.com/informacion/no...,"Xalapa, Ver",05/08/2019
259,Tour por las mejores cafeterías,09/02/2024,acompáñame a mi si si sí Wachisneris en este t...,https://www.alcalorpolitico.com/informacion/no...,,09/02/2024
4,Menor que aprendía a manejar atropelló y mató ...,"Coatzacoalcos, Ver. 16/04/2025",Una menor de edad atropelló y mató por acciden...,https://www.alcalorpolitico.com/informacion/me...,"Coatzacoalcos, Ver",16/04/2025
223,No era tornado: “Embudo de nube” sorprendió en...,"Xalapa, Ver. 16/04/2025","La tarde de este miércoles, usuarios de redes ...",https://www.alcalorpolitico.com/informacion/no...,"Xalapa, Ver",16/04/2025
...,...,...,...,...,...,...
338,"Por obra que no avanza, no pueden usar cancha ...","Xalapa, Ver. 22/04/2025",Vecinos de la colonia Revolución se manifestar...,https://www.alcalorpolitico.com/informacion/po...,"Xalapa, Ver",22/04/2025
314,Exbasquetbolista “El Diablo” Castellanos dirig...,"Veracruz, Ver. 22/04/2025",Las Rojas de Veracruz han anunciado a su entre...,https://www.alcalorpolitico.com/informacion/ex...,"Veracruz, Ver",22/04/2025
322,Captan a presunto personal de PC de Tantoyuca ...,"Xalapa, Ver. 22/04/2025",Ciudadanos del municipio de Tantoyuca denuncia...,https://www.alcalorpolitico.com/informacion/ca...,"Xalapa, Ver",22/04/2025
208,Orizabeños también reciben llamadas de estafas...,"Orizaba, Ver. 22/04/2025",En el último mes han incrementado las llamadas...,https://www.alcalorpolitico.com/informacion/or...,"Orizaba, Ver",22/04/2025


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import urljoin
import time
from concurrent.futures import ThreadPoolExecutor

# Configuración
base_url = "https://www.alcalorpolitico.com/informacion/notasarchivo.php"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_articles_from_url(target_url, target_date):
    """Extrae enlaces desde la estructura correcta (div#textonota > ol)"""
    try:
        response = requests.get(target_url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        articles = []
        contenedor = soup.find('div', id='textonota')
        
        if contenedor:
            for li in contenedor.find_all('li'):  # Cada noticia está en un <li>
                link = li.find('a')
                if link and link.get('href'):
                    full_url = urljoin(base_url, link['href'])
                    articles.append({
                        'url': full_url,
                        'fecha_scrapeo': target_date,
                        'titulo_preview': link.get_text(strip=True)[:100]
                    })
        
        print(f"📅 {target_date}: {len(articles)} artículos")
        return articles
    
    except Exception as e:
        print(f"❌ Error en {target_date}: {str(e)[:50]}...")
        return []

# Generar todas las URLs (como en tu código original)
def generate_daily_urls(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        yield {
            'url': f"{base_url}?fn={current_date.strftime('%Y-%m-%d')}",
            'date': current_date.strftime('%Y-%m-%d')
        }
        current_date += timedelta(days=1)

# Procesar todo el rango
all_articles = []
for date_info in generate_daily_urls(datetime(2024,1,1), datetime(2025,4,20)):
    all_articles.extend(get_articles_from_url(date_info['url'], date_info['date']))
    time.sleep(0.1)

if all_articles:
    print(f"✅ ¡Estructura correcta! Artículos encontrados: {len(all_articles)}")
    print("Ejemplo de URL:", all_articles[0]['url'])
else:
    print("❌ Verifica manualmente: ¿Hay noticias en las URLs de prueba?")

📅 2024-01-01: 35 artículos
📅 2024-01-02: 46 artículos
📅 2024-01-03: 47 artículos
📅 2024-01-04: 58 artículos
📅 2024-01-05: 55 artículos
📅 2024-01-06: 34 artículos
📅 2024-01-07: 29 artículos
📅 2024-01-08: 54 artículos
📅 2024-01-09: 62 artículos
📅 2024-01-10: 57 artículos
📅 2024-01-11: 58 artículos
📅 2024-01-12: 56 artículos
📅 2024-01-13: 34 artículos
📅 2024-01-14: 21 artículos
📅 2024-01-15: 63 artículos
📅 2024-01-16: 62 artículos
📅 2024-01-17: 59 artículos
📅 2024-01-18: 49 artículos
📅 2024-01-19: 72 artículos
📅 2024-01-20: 31 artículos
📅 2024-01-21: 30 artículos
📅 2024-01-22: 58 artículos
📅 2024-01-23: 49 artículos
📅 2024-01-24: 69 artículos
📅 2024-01-25: 68 artículos
📅 2024-01-26: 64 artículos
📅 2024-01-27: 36 artículos
📅 2024-01-28: 24 artículos
📅 2024-01-29: 72 artículos
📅 2024-01-30: 58 artículos
📅 2024-01-31: 70 artículos
📅 2024-02-01: 60 artículos
📅 2024-02-02: 53 artículos
📅 2024-02-03: 41 artículos
📅 2024-02-04: 31 artículos
📅 2024-02-05: 51 artículos
📅 2024-02-06: 59 artículos
📅

In [4]:
all_articles

[{'url': 'https://www.alcalorpolitico.com/informacion/arde-pastizal-en-la-ladera-del-cerro-de-las-antenas-en-nogales-ver--399809.html',
  'fecha_scrapeo': '2024-01-01',
  'titulo_preview': 'Arde pastizal en la ladera del cerro de las Antenas, en Nogales, Ver.'},
 {'url': 'https://www.alcalorpolitico.com/informacion/seis-lesionados-en-choque-entre-un-taxi-y-un-autobus-en-piletas-399808.html',
  'fecha_scrapeo': '2024-01-01',
  'titulo_preview': 'Seis lesionados en choque entre un taxi y un autobús, en Piletas'},
 {'url': 'https://www.alcalorpolitico.com/informacion/magistrada-monica-soto-asume-como-presidenta-del-tribunal-electoral-399807.html',
  'fecha_scrapeo': '2024-01-01',
  'titulo_preview': 'Magistrada Mónica Soto asume como presidenta del Tribunal Electoral'},
 {'url': 'https://www.alcalorpolitico.com/informacion/desde-palenque-amlo-desea-que-en-2024-no-haya-ninguna-calamidad-399806.html',
  'fecha_scrapeo': '2024-01-01',
  'titulo_preview': 'Desde Palenque, AMLO desea que en 20

In [5]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Función mejorada para extraer contenido (adaptada de tu scraper original)
def extract_article_data(url):
    try:
        print(f"📰 Procesando: {url[:60]}...")
        response = requests.get(url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 1. Extraer título
        title = soup.find('h1').get_text(strip=True) if soup.find('h1') else 'Sin título'
        
        # 2. Extraer fecha
        date = 'Sin fecha'
        date_element = soup.find('span', id='lugar') or soup.find('div', class_='fechapublicacion')
        if date_element:
            date_text = date_element.get_text(strip=True)
            date = date_text.split(' / ')[-1] if ' / ' in date_text else date_text
        
        # 3. Extraer contenido (versión mejorada)
        article_body = soup.find('div', class_='cuerponota') or soup.find('div', class_='cuerpoColumna')
        content = []
        
        if article_body:
            current_element = article_body.find('div', class_='google-ads') or article_body
            while current_element:
                # Manejar todos los tipos de elementos
                if isinstance(current_element, str):  # Si es texto plano
                    text = current_element.strip()
                    if text:
                        content.append(text)
                elif hasattr(current_element, 'name'):  # Si es un tag HTML
                    # Saltar elementos no deseados
                    if current_element.name in ['script', 'style', 'div', 'google-ads']:
                        current_element = current_element.next_sibling
                        continue
                    
                    # Capturar texto de párrafos o elementos con texto
                    if current_element.name == 'p' or current_element.string:
                        text = current_element.get_text(' ', strip=True)
                        if text and len(text) > 10:
                            content.append(text)
                
                # Detenerse si encontramos el marcador de fin
                if hasattr(current_element, 'get') and current_element.get('id') == 'publicidadinferior':
                    break
                    
                current_element = current_element.next_element if hasattr(current_element, 'next_element') else None
        
        # 4. Extraer autor
        author = 'Desconocido'
        author_tag = soup.find('span', class_='autor') or soup.find('p', class_='firma')
        if author_tag:
            author = author_tag.get_text(strip=True).replace('Por ', '')
        
        return {
            'titulo': title,
            'fecha': date,
            'autor': author,
            'contenido': ' '.join(content),
            'url': url,
            'longitud': len(' '.join(content))
        }
    
    except Exception as e:
        print(f"❌ Error en {url[:50]}...: {str(e)}")
        return None

# Procesamiento paralelo seguro
def scrape_articles(articles_list, max_workers=5, batch_size=50):
    results = []
    
    for i in range(0, len(articles_list), batch_size):
        batch = articles_list[i:i + batch_size]
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_url = {executor.submit(extract_article_data, article['url']): article for article in batch}
            
            for future in as_completed(future_to_url):
                result = future.result()
                if result:
                    results.append(result)
        
        print(f"✅ Lote {i//batch_size + 1}/{(len(articles_list)//batch_size)+1} completado")
        time.sleep(5)  # Pausa entre lotes para evitar bloqueos
    
    return pd.DataFrame(results)

# --- EJECUCIÓN PRINCIPAL --- #
if __name__ == "__main__":
    # 1. Convertir a DataFrame
    df_urls = pd.DataFrame(all_articles).drop_duplicates('url')
    print(f"🔍 Total de URLs únicas a scrapear: {len(df_urls)}")
    
    # 2. Scrapear contenido (ajusta los parámetros según necesidad)
    df_final = scrape_articles(
        df_urls.to_dict('records'),
        max_workers=10,  # Reduce si hay errores de conexión
        batch_size=30
    )
    
    # 3. Guardar resultados
    output_file = "noticias_alcalor_contenido.csv"
    df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n🎉 Datos guardados en '{output_file}'")
    print("📊 Resumen:")
    print(df_final[['fecha', 'titulo', 'longitud']].describe())

🔍 Total de URLs únicas a scrapear: 25007
📰 Procesando: https://www.alcalorpolitico.com/informacion/arde-pastizal-en...
📰 Procesando: https://www.alcalorpolitico.com/informacion/seis-lesionados-...
📰 Procesando: https://www.alcalorpolitico.com/informacion/magistrada-monic...
📰 Procesando: https://www.alcalorpolitico.com/informacion/desde-palenque-a...
📰 Procesando: https://www.alcalorpolitico.com/informacion/limpian-bulevar-...
📰 Procesando: https://www.alcalorpolitico.com/informacion/pese-a-promesa-d...
📰 Procesando: https://www.alcalorpolitico.com/informacion/instructora-de-o...
📰 Procesando: https://www.alcalorpolitico.com/informacion/pronostico-de-la...
📰 Procesando: https://www.alcalorpolitico.com/informacion/hacen-sandwich-a...
📰 Procesando: https://www.alcalorpolitico.com/informacion/incrementa-inseg...
📰 Procesando: https://www.alcalorpolitico.com/informacion/fin-de-anio-deja...
📰 Procesando: https://www.alcalorpolitico.com/informacion/por-quemar-el-vi...
📰 Procesando: https://w

In [6]:
df_final

,titulo,fecha,autor,contenido,url,longitud
0,Magistrada Mónica Soto asume como presidenta d...,Ciudad de México 01/01/2024,Desconocido,La magistrada Mónica Soto inició este 1° de en...,https://www.alcalorpolitico.com/informacion/ma...,2173
1,Pronóstico de las 72 horas,01/01/2024,Desconocido,Fin Texto de la nota Fin area de la nota,https://www.alcalorpolitico.com/informacion/pr...,40
2,Instructora de Orizaba lamenta “desvalorizació...,"Orizaba, Ver. 01/01/2024",Desconocido,Aunque a través de los talleres del DIF se bus...,https://www.alcalorpolitico.com/informacion/in...,1494
3,"Hacen “sándwich” a auto particular, en autopis...",Fortín de las Flores 01/01/2024,Desconocido,Daños materiales por varios miles de pesos y l...,https://www.alcalorpolitico.com/informacion/ha...,1050
4,Seis lesionados en choque entre un taxi y un a...,"Rafael Lucio, Ver. 01/01/2024",Desconocido,Seis lesionados dejó esta noche el choque entr...,https://www.alcalorpolitico.com/informacion/se...,1063
...,...,...,...,...,...,...
24955,Asesinan a balazos a taquero en la colonia El ...,"Córdoba, Ver. 20/04/2025",Desconocido,"Un taquero identificado como José Enrique ""N"",...",https://www.alcalorpolitico.com/informacion/as...,1523
24956,"Domingo caluroso; posibles tormentas, lluvias ...","Xalapa, Ver. 20/04/2025",Desconocido,"Probabilidad de lluvias, chubascos y tormentas...",https://www.alcalorpolitico.com/informacion/do...,2103
24957,La Pascua es el corazón del año litúrgico y el...,"Xalapa, Ver. 20/04/2025",Desconocido,"Este Domingo de Resurrección, el mensaje que r...",https://www.alcalorpolitico.com/informacion/la...,1613
24958,Católicos celebran el Domingo de Resurrección,"Xalapa, Ver. 20/04/2025",Desconocido,"Este 20 de abril, la comunidad católica celebr...",https://www.alcalorpolitico.com/informacion/ca...,1551


In [7]:
df_final = df_final[df_final['contenido'].str.len() > 50]  # Elimina filas con menos de 50 caracteres


In [8]:
df_final

,titulo,fecha,autor,contenido,url,longitud
0,Magistrada Mónica Soto asume como presidenta d...,Ciudad de México 01/01/2024,Desconocido,La magistrada Mónica Soto inició este 1° de en...,https://www.alcalorpolitico.com/informacion/ma...,2173
2,Instructora de Orizaba lamenta “desvalorizació...,"Orizaba, Ver. 01/01/2024",Desconocido,Aunque a través de los talleres del DIF se bus...,https://www.alcalorpolitico.com/informacion/in...,1494
3,"Hacen “sándwich” a auto particular, en autopis...",Fortín de las Flores 01/01/2024,Desconocido,Daños materiales por varios miles de pesos y l...,https://www.alcalorpolitico.com/informacion/ha...,1050
4,Seis lesionados en choque entre un taxi y un a...,"Rafael Lucio, Ver. 01/01/2024",Desconocido,Seis lesionados dejó esta noche el choque entr...,https://www.alcalorpolitico.com/informacion/se...,1063
5,Limpian bulevar Manuel Ávila Camacho en Boca d...,01/01/2024,Desconocido,Con el “barrido” sobre el bulevar Manuel Ávila...,https://www.alcalorpolitico.com/informacion/li...,374
...,...,...,...,...,...,...
24955,Asesinan a balazos a taquero en la colonia El ...,"Córdoba, Ver. 20/04/2025",Desconocido,"Un taquero identificado como José Enrique ""N"",...",https://www.alcalorpolitico.com/informacion/as...,1523
24956,"Domingo caluroso; posibles tormentas, lluvias ...","Xalapa, Ver. 20/04/2025",Desconocido,"Probabilidad de lluvias, chubascos y tormentas...",https://www.alcalorpolitico.com/informacion/do...,2103
24957,La Pascua es el corazón del año litúrgico y el...,"Xalapa, Ver. 20/04/2025",Desconocido,"Este Domingo de Resurrección, el mensaje que r...",https://www.alcalorpolitico.com/informacion/la...,1613
24958,Católicos celebran el Domingo de Resurrección,"Xalapa, Ver. 20/04/2025",Desconocido,"Este 20 de abril, la comunidad católica celebr...",https://www.alcalorpolitico.com/informacion/ca...,1551


In [9]:
import re

# Suponiendo que tu DataFrame es 'df_final' y la columna se llama 'fecha'
# Ejemplo de datos: "Orizaba, Ver. 22/04/2025"

# 1. Extraer lugar y fecha usando expresiones regulares
df_final[['lugar', 'fecha_limpia']] = df_final['fecha'].str.extract(
    r'^(.*?)\s*\.\s*(\d{2}/\d{2}/\d{4})$'  # Patrón: "texto. dd/mm/aaaa"
)

# 2. Limpieza adicional (opcional)
df_final['lugar'] = df_final['lugar'].str.strip()  # Elimina espacios en blanco
df_final['fecha_limpia'] = pd.to_datetime(df_final['fecha_limpia'], format='%d/%m/%Y')  # Convertir a datetime

# Verificar filas donde la extracción falló
filas_fallidas = df_final[df_final['fecha_limpia'].isna()]
if not filas_fallidas.empty:
    print("⚠️ Filas con formato inesperado:")
    display(filas_fallidas[['fecha']])
    
    # Opción: Extraer manualmente (ej: si algunas no tienen punto)
    df_final['fecha_limpia'] = df_final['fecha'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0]  # Solo fecha
    df_final['lugar'] = df_final['fecha'].str.replace(r'\d{2}/\d{2}/\d{4}', '', regex=True).str.strip(' .')

# 3. Mostrar resultados
print(df_final[['fecha', 'lugar', 'fecha_limpia']].head())

⚠️ Filas con formato inesperado:


/tmp/ipykernel_1849/477466535.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[['lugar', 'fecha_limpia']] = df_final['fecha'].str.extract(
/tmp/ipykernel_1849/477466535.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[['lugar', 'fecha_limpia']] = df_final['fecha'].str.extract(
/tmp/ipykernel_1849/477466535.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,fecha
0,Ciudad de México 01/01/2024
3,Fortín de las Flores 01/01/2024
5,01/01/2024
17,01/01/2024
19,01/01/2024
...,...
24918,19/04/2025
24919,Ciudad de México 19/04/2025
24924,19/04/2025
24925,19/04/2025


/tmp/ipykernel_1849/477466535.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['fecha_limpia'] = df_final['fecha'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0]  # Solo fecha


                             fecha                 lugar fecha_limpia
0      Ciudad de México 01/01/2024      Ciudad de México   01/01/2024
2         Orizaba, Ver. 01/01/2024          Orizaba, Ver   01/01/2024
3  Fortín de las Flores 01/01/2024  Fortín de las Flores   01/01/2024
4    Rafael Lucio, Ver. 01/01/2024     Rafael Lucio, Ver   01/01/2024
5                       01/01/2024                         01/01/2024


/tmp/ipykernel_1849/477466535.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['lugar'] = df_final['fecha'].str.replace(r'\d{2}/\d{2}/\d{4}', '', regex=True).str.strip(' .')


In [11]:
# Guardar a CSV
df_final.to_csv('scraped data/noticias_alcalorpolitico.csv', index=False, encoding='utf-8-sig')
print("💾 Datos guardados en 'noticias_alcalorpolitico.csv'")

# Mostrar en el notebook
from IPython.display import display
display(df_final)

💾 Datos guardados en 'noticias_alcalorpolitico.csv'


,titulo,fecha,autor,contenido,url,longitud,lugar,fecha_limpia
0,Magistrada Mónica Soto asume como presidenta d...,Ciudad de México 01/01/2024,Desconocido,La magistrada Mónica Soto inició este 1° de en...,https://www.alcalorpolitico.com/informacion/ma...,2173,Ciudad de México,01/01/2024
2,Instructora de Orizaba lamenta “desvalorizació...,"Orizaba, Ver. 01/01/2024",Desconocido,Aunque a través de los talleres del DIF se bus...,https://www.alcalorpolitico.com/informacion/in...,1494,"Orizaba, Ver",01/01/2024
3,"Hacen “sándwich” a auto particular, en autopis...",Fortín de las Flores 01/01/2024,Desconocido,Daños materiales por varios miles de pesos y l...,https://www.alcalorpolitico.com/informacion/ha...,1050,Fortín de las Flores,01/01/2024
4,Seis lesionados en choque entre un taxi y un a...,"Rafael Lucio, Ver. 01/01/2024",Desconocido,Seis lesionados dejó esta noche el choque entr...,https://www.alcalorpolitico.com/informacion/se...,1063,"Rafael Lucio, Ver",01/01/2024
5,Limpian bulevar Manuel Ávila Camacho en Boca d...,01/01/2024,Desconocido,Con el “barrido” sobre el bulevar Manuel Ávila...,https://www.alcalorpolitico.com/informacion/li...,374,,01/01/2024
...,...,...,...,...,...,...,...,...
24955,Asesinan a balazos a taquero en la colonia El ...,"Córdoba, Ver. 20/04/2025",Desconocido,"Un taquero identificado como José Enrique ""N"",...",https://www.alcalorpolitico.com/informacion/as...,1523,"Córdoba, Ver",20/04/2025
24956,"Domingo caluroso; posibles tormentas, lluvias ...","Xalapa, Ver. 20/04/2025",Desconocido,"Probabilidad de lluvias, chubascos y tormentas...",https://www.alcalorpolitico.com/informacion/do...,2103,"Xalapa, Ver",20/04/2025
24957,La Pascua es el corazón del año litúrgico y el...,"Xalapa, Ver. 20/04/2025",Desconocido,"Este Domingo de Resurrección, el mensaje que r...",https://www.alcalorpolitico.com/informacion/la...,1613,"Xalapa, Ver",20/04/2025
24958,Católicos celebran el Domingo de Resurrección,"Xalapa, Ver. 20/04/2025",Desconocido,"Este 20 de abril, la comunidad católica celebr...",https://www.alcalorpolitico.com/informacion/ca...,1551,"Xalapa, Ver",20/04/2025
